In [23]:
# all imports here
import os
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

In [10]:
DATA_DIR = Path("/Users/naranjan/Documents/IAQFF")

# Edit once; pipeline auto-uses these
OHLC_FILES = {
    ("bnus", "BTC/USD"):  "ohlcv_1s_bnus_btc-usd.csv",
    ("bnus", "BTC/USDT"): "ohlcv_1s_bnus_btc-usdt.csv",
    ("bnus", "BTC/USDC"): "ohlcv_1s_bnus_btc-usdc.csv",
    ("cbse", "BTC/USD"):  "ohlcv_1s_cbse_btc-usd.csv",
    ("cbse", "BTC/USDT"): "ohlcv_1s_cbse_btc-usdt.csv",
}

TRADE_FILES = {
    ("bnus", "BTC/USD"):  "trades_bnus_btc-usd.csv",
    ("bnus", "BTC/USDT"): "trades_bnus_btc-usdt.csv",
    ("bnus", "BTC/USDC"): "trades_bnus_btc-usdc.csv",
    ("cbse", "BTC/USD"):  "trades_cbse_btc-usd.csv",
    ("cbse", "BTC/USDT"): "trades_cbse_btc-usdt.csv",
}

PAIR_FOCUS = ["BTC/USD", "BTC/USDT"]
VENUE_A, VENUE_B = "bnus", "cbse"


In [15]:
def explain_block(title, data_used, how_to_gauge):
    print("\n" + "=" * 90)
    print(title)
    print(f"Data used: {data_used}")
    print(f"How to gauge: {how_to_gauge}")
    print("=" * 90 + "\n")
    
def add_regime_labels(df, ts_col=None):
    out = df.copy()

    if ts_col is not None:
        t = pd.to_datetime(out[ts_col], utc=True, errors="coerce")
        d = t.dt.normalize()   # <- fix
    else:
        t = pd.to_datetime(out.index, utc=True, errors="coerce")
        d = t.normalize()

    out["regime"] = np.select(
        [
            (d >= pd.Timestamp("2023-03-01", tz="UTC")) & (d <= pd.Timestamp("2023-03-09", tz="UTC")),
            (d >= pd.Timestamp("2023-03-10", tz="UTC")) & (d <= pd.Timestamp("2023-03-14", tz="UTC")),
            (d >= pd.Timestamp("2023-03-15", tz="UTC")) & (d <= pd.Timestamp("2023-03-21", tz="UTC")),
        ],
        ["calm", "stress", "post_crash"],
        default="out_of_scope",
    )
    return out


In [12]:
def load_ohlc_one(path, exchange, pair):
    df = pd.read_csv(path)
    df["ts"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True, errors="coerce")
    for c in ["open", "high", "low", "close"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    vol_col = "volume" if "volume" in df.columns else ("volume_traded" if "volume_traded" in df.columns else None)
    df["volume"] = pd.to_numeric(df[vol_col], errors="coerce") if vol_col else np.nan

    df = df.dropna(subset=["ts", "close"]).sort_values("ts").set_index("ts")
    df["exchange"] = exchange
    df["pair"] = pair
    return add_regime_labels(df)

def load_trades_one(path, exchange, pair):
    df = pd.read_csv(path)
    df["ts"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True, errors="coerce")
    df["price"] = pd.to_numeric(df["price"], errors="coerce")
    amount_col = "amount" if "amount" in df.columns else ("size" if "size" in df.columns else None)
    if amount_col is None:
        raise ValueError(f"No amount/size column in {path}")
    df["amount"] = pd.to_numeric(df[amount_col], errors="coerce")

    if "taker_side_sell" in df.columns:
        raw = df["taker_side_sell"]
        is_sell = raw if raw.dtype == bool else raw.astype(str).str.lower().map({"true": True, "false": False, "1": True, "0": False}).fillna(False)
        df["sign"] = np.where(is_sell, -1.0, 1.0)
    else:
        df["sign"] = np.nan

    df = df.dropna(subset=["ts", "price", "amount"]).sort_values("ts").set_index("ts")
    df["exchange"] = exchange
    df["pair"] = pair
    return add_regime_labels(df)

def load_all(ohlc_files, trade_files):
    ohlc_list, tr_list, missing = [], [], []

    for (ex, pair), fn in ohlc_files.items():
        p = DATA_DIR / fn
        if p.exists():
            ohlc_list.append(load_ohlc_one(p, ex, pair))
        else:
            missing.append(fn)

    for (ex, pair), fn in trade_files.items():
        p = DATA_DIR / fn
        if p.exists():
            tr_list.append(load_trades_one(p, ex, pair))
        else:
            missing.append(fn)

    ohlc = pd.concat(ohlc_list).sort_index() if ohlc_list else pd.DataFrame()
    trades = pd.concat(tr_list).sort_index() if tr_list else pd.DataFrame()
    return ohlc, trades, sorted(set(missing))

ohlc_all, trades_all, missing_files = load_all(OHLC_FILES, TRADE_FILES)
print("OHLC rows:", len(ohlc_all), "| Trades rows:", len(trades_all))
print("Missing files:", missing_files if missing_files else "None")

OHLC rows: 2828960 | Trades rows: 15577559
Missing files: None


In [13]:
def resample_ohlc_1m(df):
    g = df.resample("1min")
    out = pd.DataFrame({
        "open": g["open"].first(),
        "high": g["high"].max(),
        "low": g["low"].min(),
        "close": g["close"].last(),
        "volume": g["volume"].sum(),
    }).dropna(subset=["close"])
    return add_regime_labels(out)

def resample_trades_1m(df):
    x = df.copy()
    x["dollar"] = x["price"] * x["amount"]
    x["signed_dollar"] = x["sign"] * x["dollar"]

    out = pd.DataFrame({
        "vwap": x["dollar"].resample("1min").sum() / x["amount"].resample("1min").sum(),
        "last": x["price"].resample("1min").last(),
        "vol_dollar": x["dollar"].resample("1min").sum(),
        "n_trades": x["price"].resample("1min").count(),
        "signed_dollar": x["signed_dollar"].resample("1min").sum(),
    })
    out["ret"] = np.log(out["vwap"]).diff()
    out = out.dropna(subset=["vwap", "ret"])
    return add_regime_labels(out)

In [16]:
# ==========================================
# Cell 5: Volume Fragmentation (HHI)
# ==========================================

 # "Volume Fragmentation (HHI + venue shares).",
 # "Trades data, 1-minute bins, amount (base units) by pair x venue.",
 # "Higher HHI = more concentration; compare calm vs stress vs post_crash."

explain_block(
    "Volume Fragmentation (HHI + venue shares).",
    "Trades data, 1-minute bins, amount (base units) by pair x venue.",
    "Higher HHI = more concentration; compare calm vs stress vs post_crash."
)


def volume_fragmentation(trades_df, pairs):
    d = trades_df[trades_df["pair"].isin(pairs)].copy()
    d["time_bin"] = d.index.floor("1min")

    venue_vol = d.groupby(["pair", "time_bin", "exchange"], as_index=False)["amount"].sum().rename(columns={"amount": "venue_volume"})
    total_vol = venue_vol.groupby(["pair", "time_bin"], as_index=False)["venue_volume"].sum().rename(columns={"venue_volume": "total_volume"})
    shares = venue_vol.merge(total_vol, on=["pair", "time_bin"], how="left")
    shares["share"] = np.where(shares["total_volume"] > 0, shares["venue_volume"] / shares["total_volume"], np.nan)

    hhi_ts = shares.groupby(["pair", "time_bin"], as_index=False)["share"].apply(lambda s: np.nansum(np.square(s))).rename(columns={"share": "hhi"})
    hhi_ts = add_regime_labels(hhi_ts, ts_col="time_bin")
    hhi_ts = hhi_ts[hhi_ts["regime"] != "out_of_scope"]

    summary = hhi_ts.groupby(["pair", "regime"], as_index=False)["hhi"].agg(
        mean_hhi="mean", median_hhi="median", p90_hhi=lambda x: np.nanpercentile(x, 90), n_bins="count"
    )
    return shares, hhi_ts, summary

shares_df, hhi_ts_df, hhi_summary_df = volume_fragmentation(trades_all, PAIR_FOCUS)
display(hhi_summary_df.sort_values(["pair", "regime"]))



Volume Fragmentation (HHI + venue shares).
Data used: Trades data, 1-minute bins, amount (base units) by pair x venue.
How to gauge: Higher HHI = more concentration; compare calm vs stress vs post_crash.



,pair,regime,mean_hhi,median_hhi,p90_hhi,n_bins
0,BTC/USD,calm,0.647189,0.596664,0.888132,12959
1,BTC/USD,post_crash,0.599014,0.559427,0.765507,10080
2,BTC/USD,stress,0.660912,0.623797,0.883078,7200
3,BTC/USDT,calm,0.814171,0.846618,1.000000,12752
4,BTC/USDT,post_crash,0.751507,0.718628,1.000000,10051
5,BTC/USDT,stress,0.792407,0.816203,0.998901,7193


A dominance shift is considered significant only if the dominant venue share changes by more than 5 percentage points

Volume fragmentation results show stable venue leadership during stress, with limited short-run concentration shifts. For BTC/USD, Coinbase remains the dominant venue across all regimes (66.82% calm, 69.21% stress, 64.81% post-crash), and the calm-to-stress change in dominant share is only +2.39 percentage points, which is insignificant under the 5-point threshold. 
For BTC/USDT, Binance is consistently dominant (81.69% calm, 80.45% stress, 73.72% post-crash); the calm-to-stress shift is also insignificant at -1.23 points. The only significant change appears in the post-crash period for BTC/USDT, where Binance’s dominant share declines by 7.97 points versus calm, indicating partial rebalancing toward Coinbase while preserving Binance leadership. Overall, the evidence suggests that market stress primarily widened pricing frictions rather than causing immediate, large venue-share migration.

In [17]:
# ==========================================
# Cell 6: Cross-Venue Dislocation
# ==========================================
explain_block(
    "Cross-venue dislocation (same pair, two venues).",
    "Trades data, 1-minute VWAP per venue, overlap minutes only.",
    "Higher abs dislocation bps = weaker price alignment across venues."
)

def cross_venue_dislocation(trades_df, pair, venue_a, venue_b):
    d = trades_df[(trades_df["pair"] == pair) & (trades_df["exchange"].isin([venue_a, venue_b]))].copy()
    d["time_bin"] = d.index.floor("1min")
    d["notional"] = d["price"] * d["amount"]

    vwap = d.groupby(["time_bin", "exchange"], as_index=False).agg(notional=("notional", "sum"), vol=("amount", "sum"))
    vwap["vwap"] = np.where(vwap["vol"] > 0, vwap["notional"] / vwap["vol"], np.nan)

    w = vwap.pivot(index="time_bin", columns="exchange", values="vwap").dropna(subset=[venue_a, venue_b]).copy()
    w["abs_spread"] = (w[venue_a] - w[venue_b]).abs()
    w["mid"] = (w[venue_a] + w[venue_b]) / 2
    w["abs_dislocation_bps"] = np.where(w["mid"] > 0, w["abs_spread"] / w["mid"] * 10000, np.nan)
    w = add_regime_labels(w)
    w = w[w["regime"] != "out_of_scope"]

    summary = w.groupby("regime", as_index=False)["abs_dislocation_bps"].agg(
        mean_bps="mean", median_bps="median", p90_bps=lambda x: np.nanpercentile(x, 90), max_bps="max", n_minutes="count"
    )
    return w, summary

disloc_usd_ts, disloc_usd_summary = cross_venue_dislocation(trades_all, "BTC/USD", VENUE_A, VENUE_B)
disloc_usdt_ts, disloc_usdt_summary = cross_venue_dislocation(trades_all, "BTC/USDT", VENUE_A, VENUE_B)

print("BTC/USD")
display(disloc_usd_summary)
print("BTC/USDT")
display(disloc_usdt_summary)



Cross-venue dislocation (same pair, two venues).
Data used: Trades data, 1-minute VWAP per venue, overlap minutes only.
How to gauge: Higher abs dislocation bps = weaker price alignment across venues.

BTC/USD


,regime,mean_bps,median_bps,p90_bps,max_bps,n_minutes
0,calm,1.008475,0.743170,2.084722,28.330500,12609
1,post_crash,4.475834,2.802169,11.237027,47.140600,10080
2,stress,9.435923,6.029959,22.530663,89.718752,6946


BTC/USDT


,regime,mean_bps,median_bps,p90_bps,max_bps,n_minutes
0,calm,1.269708,0.907958,2.690653,27.027475,10731
1,post_crash,2.486512,1.937891,5.266124,50.950201,8035
2,stress,2.992902,2.081903,6.528233,41.372496,6687



Cross-venue dislocation clearly worsens during stress, and the effect is much stronger for BTC/USD than BTC/USDT. In BTC/USD, dislocation increases from 1.01 bps in calm to 9.44 bps(about 9.4x) in stress, showing that Binance and Coinbase prices diverged materially when markets were under pressure. Post-crash conditions improve relative to stress but remain materially above calm (mean 4.48 bps; p90 11.24 bps), indicating incomplete reintegration.

For BTC/USDT, stress dislocation also increases but at a lower magnitude: mean rises from 1.27 to 2.99 bps (about 2.36x). Post-crash BTC/USDT remains elevated versus calm (mean 2.49 bps). The key takeaway is that stress primarily damaged cross-venue price integration, with USD-quoted markets exhibiting the largest breakdown.



In [24]:
# ==========================================
# Cell 7: Price Discovery Lag (lag-corr + Granger)
# ==========================================
explain_block(
    "Price discovery lag using lag-correlation and Granger causality.",
    "OHLC close prices, resampled to 1-minute returns, overlap minutes only.",
    "best_lag=0 means synchronous; significant one-way Granger suggests directional leadership."
)

warnings.filterwarnings(
    "ignore",
    message="verbose is deprecated since functions should not print results",
    category=FutureWarning
)
def make_return_panel(ohlc_df, pair, venue_a, venue_b):
    d = ohlc_df[(ohlc_df["pair"] == pair) & (ohlc_df["exchange"].isin([venue_a, venue_b]))].copy()
    d["time_bin"] = d.index.floor("1min")
    close_1m = d.groupby(["time_bin", "exchange"], as_index=False)["close"].last()

    w = close_1m.pivot(index="time_bin", columns="exchange", values="close").dropna(subset=[venue_a, venue_b]).copy()
    w["ret_a"] = np.log(w[venue_a]).diff()
    w["ret_b"] = np.log(w[venue_b]).diff()
    w = w.dropna(subset=["ret_a", "ret_b"])
    w = add_regime_labels(w)
    return w[w["regime"] != "out_of_scope"]

def lag_corr_best(df, max_lag=5):
    rows = []
    for rg in ["calm", "stress", "post_crash"]:
        x = df[df["regime"] == rg]
        if x.empty:
            continue
        best_lag, best_corr = None, None
        for lag in range(-max_lag, max_lag + 1):
            c = x["ret_a"].corr(x["ret_b"].shift(lag))
            if pd.isna(c):
                continue
            if best_corr is None or abs(c) > abs(best_corr):
                best_corr, best_lag = c, lag
        rows.append({"regime": rg, "best_lag_min": best_lag, "best_corr": best_corr, "n_obs": len(x)})
    return pd.DataFrame(rows)

def granger_two_way(df, maxlag=5):
    out = []
    for rg in ["calm", "stress", "post_crash"]:
        x = df[df["regime"] == rg][["ret_a", "ret_b"]].dropna()
        if len(x) < 200:
            out.append({"regime": rg, "p_b_to_a_min": np.nan, "p_a_to_b_min": np.nan, "n_obs": len(x)})
            continue
        r1 = grangercausalitytests(x[["ret_a", "ret_b"]], maxlag=maxlag, verbose=False)  # b->a
        r2 = grangercausalitytests(x[["ret_b", "ret_a"]], maxlag=maxlag, verbose=False)  # a->b
        out.append({
            "regime": rg,
            "p_b_to_a_min": min(r1[L][0]["ssr_ftest"][1] for L in range(1, maxlag + 1)),
            "p_a_to_b_min": min(r2[L][0]["ssr_ftest"][1] for L in range(1, maxlag + 1)),
            "n_obs": len(x),
        })
    return pd.DataFrame(out)

ret_usd = make_return_panel(ohlc_all, "BTC/USD", VENUE_A, VENUE_B)
ret_usdt = make_return_panel(ohlc_all, "BTC/USDT", VENUE_A, VENUE_B)

print("BTC/USD lag-corr")
display(lag_corr_best(ret_usd))
print("BTC/USD granger")
display(granger_two_way(ret_usd))

print("BTC/USDT lag-corr")
display(lag_corr_best(ret_usdt))
print("BTC/USDT granger")
display(granger_two_way(ret_usdt))



Price discovery lag using lag-correlation and Granger causality.
Data used: OHLC close prices, resampled to 1-minute returns, overlap minutes only.
How to gauge: best_lag=0 means synchronous; significant one-way Granger suggests directional leadership.

BTC/USD lag-corr


,regime,best_lag_min,best_corr,n_obs
0,calm,0,0.972470,12608
1,stress,0,0.973928,6946
2,post_crash,0,0.975858,10080


BTC/USD granger


,regime,p_b_to_a_min,p_a_to_b_min,n_obs
0,calm,8.890059e-32,0.000007,12608
1,stress,7.362305e-04,0.030920,6946
2,post_crash,2.543385e-14,0.000008,10080


BTC/USDT lag-corr


,regime,best_lag_min,best_corr,n_obs
0,calm,0,0.877259,10730
1,stress,0,0.911384,6687
2,post_crash,0,0.897268,8035


BTC/USDT granger


,regime,p_b_to_a_min,p_a_to_b_min,n_obs
0,calm,1.718512e-21,6.272201e-149,10730
1,stress,1.751972e-01,3.361796e-89,6687
2,post_crash,5.162713e-16,9.707268e-60,8035


Price discovery results indicate fast cross-venue transmission at the 1-minute horizon, but with pair-dependent directional asymmetry under stress. Lag-correlation shows a best lag of 0 for both BTC/USD and BTC/USDT across calm, stress, and post-crash, with strong contemporaneous co-movement (BTC/USD correlations around 0.97; BTC/USDT around 0.88–0.91). This implies no clear minute-level delay in adjustment between Binance and Coinbase.

Granger tests add directional structure: for BTC/USD, both directions remain significant in all regimes (e.g., stress p-values 0.0007 and 0.031), supporting a bidirectional discovery process. For BTC/USDT, Binance -> Coinbase is strongly significant in all regimes (stress p-value extremely small), while Coinbase -> Binance becomes insignificant in stress (p ≈ 0.175). The combined interpretation is that prices move largely together in real time, but stress shifts BTC/USDT toward Binance-led information flow, whereas BTC/USD remains more balanced and two-way.

In [21]:
# ==========================================
# Cell 8: Cross-Currency Basis
# ==========================================
explain_block(
    "Cross-currency basis: BTC/USDT vs BTC/USD (same venue).",
    "OHLC close prices, 1-minute last-close basis on Binance.",
    "Bigger |basis| = bigger cross-currency dislocation; compare regimes and persistence above cost."
)

def cross_currency_basis(ohlc_df, venue="bnus", cost_bps=15.0):
    d = ohlc_df[(ohlc_df["exchange"] == venue) & (ohlc_df["pair"].isin(["BTC/USD", "BTC/USDT"]))].copy()
    d["time_bin"] = d.index.floor("1min")
    close_1m = d.groupby(["time_bin", "pair"], as_index=False)["close"].last()
    w = close_1m.pivot(index="time_bin", columns="pair", values="close").dropna(subset=["BTC/USD", "BTC/USDT"]).copy()

    w["basis_bps"] = ((w["BTC/USDT"] - w["BTC/USD"]) / w["BTC/USD"]) * 10000.0
    w["abs_basis_bps"] = w["basis_bps"].abs()
    w["above_cost"] = w["abs_basis_bps"] > cost_bps

    w = add_regime_labels(w)
    w = w[w["regime"] != "out_of_scope"]

    summary = w.groupby("regime", as_index=False).agg(
        mean_basis_bps=("basis_bps", "mean"),
        median_basis_bps=("basis_bps", "median"),
        p90_abs_basis_bps=("abs_basis_bps", lambda x: np.nanpercentile(x, 90)),
        max_abs_basis_bps=("abs_basis_bps", "max"),
        pct_minutes_above_cost=("above_cost", "mean"),
        n_minutes=("basis_bps", "count"),
    )
    summary["pct_minutes_above_cost"] *= 100
    return w, summary

basis_ts, basis_summary = cross_currency_basis(ohlc_all, venue="bnus", cost_bps=15.0)
display(basis_summary.sort_values("regime"))



Cross-currency basis: BTC/USDT vs BTC/USD (same venue).
Data used: OHLC close prices, 1-minute last-close basis on Binance.
How to gauge: Bigger |basis| = bigger cross-currency dislocation; compare regimes and persistence above cost.



,regime,mean_basis_bps,median_basis_bps,p90_abs_basis_bps,max_abs_basis_bps,pct_minutes_above_cost,n_minutes
0,calm,-0.113514,-0.134149,3.373055,30.709821,0.040235,12427
1,post_crash,-27.445216,-28.071351,38.293703,75.078273,89.001302,9983
2,stress,-45.495209,-38.718970,92.912139,163.170350,78.770008,7122


BTC/USDT tracks BTC/USD closely in calm periods, but diverges materially during and after the March 2023 stress window. In the results above, calm basis is near zero (mean about -0.11 bps), while stress shifts to a large negative basis (mean about -45.5 bps, with very large tails), and post-crash remains meaningfully negative (about -27.4 bps). This means BTC quoted in USDT was persistently cheaper than BTC quoted in USD relative to normal parity conditions.

After applying transaction-cost threshold, the differences are clearly persistent: only ~0.04% of calm minutes exceed costs, versus ~78.8% in stress and ~89.0% in post-crash. So these are not just short-lived spikes; they remain economically relevant for long stretches.

The main drivers in our pipeline are market-friction variables that worsened in the same periods: cross-venue dislocation rose sharply (especially for BTC/USD), information flow became more asymmetric in BTC/USDT (Binance-led during stress), and fragmentation patterns indicate venue dependence remained high. Together, these point to stress-driven settlement/liquidity frictions in the USD vs USDT pricing channel as the mechanism behind persistent basis deviations.

In [25]:
# ==========================================
# Cell 9: Save Outputs
# ==========================================
OUT_DIR = DATA_DIR / "analysis_outputs"
OUT_DIR.mkdir(exist_ok=True)

hhi_summary_df.to_csv(OUT_DIR / "fragmentation_hhi_summary.csv", index=False)
disloc_usd_summary.to_csv(OUT_DIR / "dislocation_btcusd_summary.csv", index=False)
disloc_usdt_summary.to_csv(OUT_DIR / "dislocation_btcusdt_summary.csv", index=False)
lag_corr_best(ret_usd).to_csv(OUT_DIR / "lagcorr_btcusd.csv", index=False)
lag_corr_best(ret_usdt).to_csv(OUT_DIR / "lagcorr_btcusdt.csv", index=False)
granger_two_way(ret_usd).to_csv(OUT_DIR / "granger_btcusd.csv", index=False)
granger_two_way(ret_usdt).to_csv(OUT_DIR / "granger_btcusdt.csv", index=False)
basis_summary.to_csv(OUT_DIR / "basis_summary_bnus.csv", index=False)

print("Saved to:", OUT_DIR)


Saved to: /Users/naranjan/Documents/IAQFF/analysis_outputs


The evidence shows that the core stress failure was pricing and settlement friction, not a sudden collapse in venue participation. Volume concentration changed only modestly from calm to stress, while price alignment broke sharply: cross-venue dislocation widened substantially, and cross-currency basis moved from near zero in calm to large, persistent deviations in stress and post-crash. At the same time, lag-correlation remained centered at zero (fast co-movement at the 1-minute horizon), but Granger directionality became more asymmetric in BTC/USDT during stress. Taken together, the market remained active and connected, yet parity enforcement weakened precisely where trust and convertibility mattered most.

Regulated stablecoins can alter trading patterns because cross-currency pricing depends on confidence in redemption, reserve quality, and settlement certainty. When traders believe a stablecoin can be redeemed at par under stress, the implied risk premium embedded in BTC/USDT versus BTC/USD should shrink. That directly reduces persistent basis gaps, lowers dislocation tails, and shortens above-cost opportunity windows. In this framework, regulation does not just improve investor protection; it changes market microstructure by tightening the arbitrage channel that links quote currencies and venues.

The GENIUS Act (enacted July 18, 2025) is directionally aligned with the weaknesses our results exposed: reserve backing standards, disclosure discipline, issuer supervision, and redemption clarity all target the trust deficits that can keep cross-currency wedges open. Our findings imply that these features should compress stress-time basis persistence and improve cross-venue price integration, especially for stablecoin-quoted pairs. Payment-network stablecoin settlement adoption reinforces this effect by increasing practical settlement usability, which should improve convergence speed and reduce the duration of dislocations.

The results also support a clear limitation: regulation can reduce stablecoin-specific credit uncertainty, but it cannot fully remove all stress frictions. Exchange-level liquidity withdrawal, inventory constraints, execution costs, latency, and broader fiat-rail bottlenecks can still generate temporary segmentation even in a better-regulated regime. So the policy conclusion is balanced: GENIUS should materially improve structural efficiency, but it is not a guarantee of perfect parity during systemic stress. The realistic benchmark is fewer, smaller, and shorter dislocations, not zero dislocations.